In [3]:
import noisy_load_profiles as nolp
from noisy_load_profiles import Pipeline
from noisy_load_profiles.perturbations import MultiplicativeGaussianNoise, ZeroMeasurements
import numpy as np

In [4]:
print("All available perturbations:")
print(dir(nolp.perturbations))

All available perturbations:
['AdditiveOUNoise', 'ConstantRandomPercentualBias', 'ConstantRandomPercentualScaling', 'DiscreteTimeShift', 'MultiplicativeGaussianNoise', 'PercentualDeadBand', 'ZeroMeasurements', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'measurement', 'random', 'systematic']


In [5]:

timesteps = 10
n_profiles = 2
profiles = np.ones((timesteps, n_profiles)) # 2 profiles with 10 timesteps each; example

gaussian_noise = MultiplicativeGaussianNoise(mean=0.0, std=0.01, seed=42)
pipeline = Pipeline([gaussian_noise])

perturbed_profiles = pipeline.perturb(profiles)
print("Original Profiles:\n", profiles)
print("Perturbed Profiles:\n", perturbed_profiles)

print("Transformation Parameters:\n", pipeline.get_transformations())

Original Profiles:
 [[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Perturbed Profiles:
 [[1.00496714 0.99861736]
 [1.00647689 1.0152303 ]
 [0.99765847 0.99765863]
 [1.01579213 1.00767435]
 [0.99530526 1.0054256 ]
 [0.99536582 0.9953427 ]
 [1.00241962 0.9808672 ]
 [0.98275082 0.99437712]
 [0.98987169 1.00314247]
 [0.99091976 0.98587696]]
Transformation Parameters:
 {MultiplicativeGaussianNoise(mean=0.0, std=0.01): {'noise_samples': array([[ 0.00496714, -0.00138264],
       [ 0.00647689,  0.0152303 ],
       [-0.00234153, -0.00234137],
       [ 0.01579213,  0.00767435],
       [-0.00469474,  0.0054256 ],
       [-0.00463418, -0.0046573 ],
       [ 0.00241962, -0.0191328 ],
       [-0.01724918, -0.00562288],
       [-0.01012831,  0.00314247],
       [-0.00908024, -0.01412304]]), 'mean': 0.0, 'std': 0.01, 'shape': (10, 2)}}


In [7]:
pert = ZeroMeasurements(f_zero=0.05, k_max=10, tail_eps=0.1, seed=42)


In [8]:

#gaussian_noise = MultiplicativeGaussianNoise(mean=0.0, std=0.01, seed=42)
pipeline = Pipeline([pert])

perturbed_profiles = pipeline.perturb(profiles)
print("Original Profiles:\n", profiles)
print("Perturbed Profiles:\n", perturbed_profiles)

print("Transformation Parameters:\n", pipeline.get_transformations())

Original Profiles:
 [[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Perturbed Profiles:
 [[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Transformation Parameters:
 {ZeroMeasurements(alpha=None, beta=None, f_zero=0.05, k_max=10, tail_eps=0.1, per_column_independent=True): {'zero_mask': array([[False, False],
       [False, False],
       [False, False],
       [False, False],
       [False, False],
       [False, False],
       [False, False],
       [False, False],
       [False, False],
       [False, False]]), 'alpha': 0.010824829751353607, 'beta': 0.2056717652757185, 'f_zero_implied': 0.0, 'run_stats': {'n_runs_per_column': [0, 0], 'run_lengths_per_column': [[], []]}, 'shape': (10, 2), 'per_column_independent': True}}
